In [ ]:
pip install --upgrade pip

In [ ]:
# Install requirements
!pip install -r Requirements.txt

In [ ]:
# View AWS Configuration
!aws configure list

----Code Start----

In [ ]:
# Imports 
from ray.util import inspect_serializability
import ray
import pyarrow.fs as pq
import pandas as pd

In [ ]:
# Reusable definitions here
bucket = 'DH-SECURE-THANOS-RAY-USE'
endpoint = 'https://s3.upshift.redhat.com'

year = '2021'
month = '01'
day = '01'

Following cells reads one days worth of parquet files and writes back a single parquet file
Month 1, Day 1.
1. Fails currently because worker nodes keep dying.
2. Success! Successful compaction for a days worth of parquets. Next is a month.

In [ ]:
# Read path
read_path = 'raydev/metric=cluster_version'
read_bucket_uri = f's3://{bucket}/{read_path}/year={year}/month={month}/day={day}'
print(read_bucket_uri)

# Write path
write_path = 'raydev-write-demo/metric=cluster_version'
write_bucket_uri = f's3://{bucket}/{write_path}/year={year}/month={month}/day={day}'

In [ ]:
# S3FileSystem in PyArrow
fs_pyarrow = pq.S3FileSystem(endpoint_override=endpoint)

# Reading parquet using Ray through filesystem
df = ray.data.read_parquet(paths=read_bucket_uri, filesystem=fs_pyarrow)

# Writing back a single parquet file
df.repartition(1).write_parquet(path=write_bucket_uri, filesystem=fs_pyarrow)

Following cells try compaction for a whole month:
1. Reading from multiple directories seems to be not supported
2. Union of dataframes is supported, can be done before the compaction step. https://github.com/ray-project/ray/issues/24598
Status: Fails due to memory limits, cannot load a dataframe for more than several days worth of parquets

In [ ]:
# Read path
read_path = 'raydev/metric=cluster_version'
read_bucket_uri = f's3://{bucket}/{read_path}/year={year}/month={month}/day={day}'
# Holds paths 
read_bucket_multiple = []

# Creating paths for every day in the month.
for i in range(2,32):
    if i < 10:
        day = '0'+str(i)
    else:
        day = str(i)
    
    read_bucket_multiple.append(f"s3://{bucket}/{read_path}/year={year}/month={month}/day={day}")
                           
print(read_bucket_multiple)


# Write path
write_path = 'raydev-write-demo/metric=cluster_version'
write_bucket_uri = f's3://{bucket}/{write_path}/year={year}/month={month}'


In [ ]:
# S3FileSystem in PyArrow
fs_pyarrow = pq.S3FileSystem(endpoint_override=endpoint)


# Initial read
master_df = ray.data.read_parquet(paths='s3://DH-SECURE-THANOS-RAY-USE/raydev/metric=cluster_version/year=2021/month=01/day=01', filesystem=fs_pyarrow)


for x in read_bucket_multiple:
    df = ray.data.read_parquet(paths=x, filesystem=fs_pyarrow) # Reading parquets for each day using Ray through filesystem

    master_df = master_df.union(df) # Followed by unioning the dataframe each time
    


In [ ]:
# Writing back a single parquet file
df.repartition(1).write_parquet(path=write_bucket_uri, filesystem=fs_pyarrow)